# Mapping DANA 2019 Flooding and Sediment Plume in Mar Menor using Sentinel-1 SAR

This notebook detects flooded areas and highlights sediment plumes caused by the September 2019 DANA event in the Mar Menor region using Sentinel-1 VV backscatter. It creates before/after median composites, identifies water using a threshold, computes reflectivity change to reveal sediment-rich areas, and visualizes results on an interactive map. Earth Engine calls may take time.

In [1]:
# Imports and Earth Engine initialization
import ee
import geemap

# Initialize Earth Engine (authenticate if needed)
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()


In [2]:
# Analysis configuration: event periods and AOI
# DANA peak was around 12-14 Sep 2019; we define before/after windows
before_start = '2019-09-01'
before_end = '2019-09-10'
after_start = '2019-09-14'
after_end = '2019-09-21'

# Area of interest covering Campo de Cartagena and the Mar Menor (polygon)
aoi = ee.Geometry.Polygon([
    [-0.7525010397061688, 37.88948999937322],
    [-0.8060344170800802, 37.89141594762948],
    [-0.9369668300256822, 37.84237282648271],
    [-1.0082296404312558, 37.769738583926646],
    [-1.0376769713508793, 37.74073202619191],
    [-1.0341435939769678, 37.68720630363401],
    [-1.0102841247516163, 37.65777126035968],
    [-0.9189328193280476, 37.55604663597647],
    [-0.718559540295739, 37.60522151419349],
    [-0.6905554748006199, 37.63584518455255],
])

# Map display parameters
map_center = [37.7, -0.95]
zoom_level = 11


In [3]:
# Load and filter Sentinel-1 collection (VV polarization, IW mode)
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(aoi)
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.eq('instrumentMode', 'IW')))

# Quick check (optional): print collection size
# print('Sentinel-1 scenes in AOI:', sentinel1.size().getInfo())


In [4]:
# Create median composites for before and after periods and clip to the AOI
median_before = sentinel1.filterDate(before_start, before_end).median().clip(aoi)
median_after = sentinel1.filterDate(after_start, after_end).median().clip(aoi)

# Visualization params for VV backscatter (grayscale)
vv_vis = {'min': -25, 'max': 0}


### Flood detection (water mask)
Water surfaces usually return low VV backscatter; a simple threshold can detect calm water bodies. The threshold here is a heuristic and may need tuning for different dates or incidence angles.

In [5]:
# Flood detection using a threshold on the 'after' composite
water_threshold = -16  # dB, heuristic
flood_mask = median_after.select('VV').lt(water_threshold).selfMask()

# Water layer visualization (blue)
water_vis = {'palette': ['0000FF']}

### Sediment plume (reflectivity change)
Sediment-laden water tends to show increased radar backscatter (more roughness/returns). We compute after - before on VV to highlight areas with increased reflectivity likely related to sediments.

In [6]:
# Compute VV reflectivity change (after - before)
vv_change = median_after.select('VV').subtract(median_before.select('VV'))

# Visualization for change: black->white->red (increase shows in red)
change_vis = {'min': -3, 'max': 10, 'palette': ['000000', 'FFFFFF', 'FF0000']}

In [7]:
# Interactive map: show VV composites, flood mask and VV change
m = geemap.Map(center=map_center, zoom=zoom_level, height='800px', width='100%')
m.add_basemap('SATELLITE')
# Add before/after VV composites for context (optional)
m.addLayer(median_before.select('VV'), vv_vis, 'VV - before')
m.addLayer(median_after.select('VV'), vv_vis, 'VV - after')
# Add flood mask and change layer
m.addLayer(flood_mask, water_vis, 'Flooded areas (heuristic)')
m.addLayer(vv_change, change_vis, 'VV change (after - before)')
m.addLayerControl()
display(m)


Map(center=[37.7, -0.95], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright',…

Color code interpretation (VV change: after - before):
- Dark/black: areas where radar reflectivity decreased or remained stable; on land this indicates little impact or rapid drainage, while over water it may indicate calm, clear water.
- White/light gray: moderate increase in reflectivity.
- Red: strong and significant increase in reflectivity after the DANA; this is the primary indicator of event impact.

Key observations visible in the map:
1) Basin impact (Rambla del Albujón and Campo de Cartagena): the basin area shows a characteristic severe flood pattern with scattered red pixels indicating widespread soil saturation and small pools of water and mud. Linear dark features (roads, parcel boundaries) contrast with surrounding red areas, suggesting some infrastructure or elevated zones remained less affected or drained faster.
2) Sediment plume in the Mar Menor: at the Albujón mouth there is a dense red concentration extending from the coast into the lagoon. This red patch over water indicates the sediment plume — a large discharge of turbid, sediment-laden freshwater that increased surface roughness and radar backscatter, clearly delimiting the plume's initial extent after the DANA.

### Notes and next steps
This pipeline uses simple heuristics (thresholds and median composites) that work as a fast first pass for a prototype. For production use we should consider: incidence-angle normalization, speckle filtering, VV/VH ratios or multi-temporal change detection algorithms, and validation with in-situ or optical imagery.